This notebook goes through how to draw a ***candy cane*** by **bending** a **segment**.

# IMPORTS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spomso.cores.helper_functions import generate_grid, smarter_reshape, hard_binarization
from spomso.cores.geom_2d import Segment

# PARAMETERS

In [ ]:
# size of the volume
co_size = 3, 7
# resolution of the volume
co_resolution = 300, 700

show = "FIELD" # BINARY, FIELD
show_midplane = True

# COORDINATE SYSTEM

Point cloud of coordinates is created with the **generate_grid** function, which takes the size of the coordinate system and the number of points in each direction (**co_resolution**).

In [ ]:
coor, co_res_new = generate_grid(co_size, co_resolution)

If there is an even number of points along any direction the function will automatically convert it to an odd number, therefore the origin is always included in the point cloud. The point cloud is a numpy.ndarray with a shape of **(D, N)**, where **D** is the number of dimensions and **N** is the number of data points.

# CREATE SDFs

Create the **line/segment** with a length of **6.5**:

In [ ]:
seg = Segment((-5, 0, 0), (1.5, 0, 0))

Give the line a thickness of **0.15**:

In [ ]:
seg.rounding(0.15)

**Bend** the **line** around the origin with the bending radius of **0.5** and for an angle of **π**:

In [ ]:
seg.bend(0.5, np.pi)

**Rotate** and **position** the ***candy cane*** in space:

In [ ]:
seg.rotate(np.pi, (0, 0, 1))
seg.move((0, 2.5, 0))

## EVALUATE

Evaluate the SDF of the ***candy cane*** to create a signed distance field 2D map. This is done by calling the **create** method and passing in the point cloud of coordinates:

In [ ]:
candy_cane_pattern = seg.create(coor)

# BINARIZATION

Convert the distance field to a binary voxel map, where 1 corresponds to the interior and 0 to the exterior of the geometry.

In [ ]:
if show_midplane:
    field = smarter_reshape(candy_cane_pattern, co_resolution)
    if show=="BINARY":
        pattern_2d = hard_binarization(field, 0)

if show=="BINARY":
    pattern = hard_binarization(candy_cane_pattern, 0)

# PLOT

In [ ]:
print("Drawing results...")
# Mid-plane cross-section plot
if show_midplane and show=="BINARY":
    fig, ax = plt.subplots(1,1, figsize=(8.25, 8.25))
    ax.imshow(pattern_2d[:, :].T,
              cmap="binary_r",
              extent=(-co_size[0]/2, co_size[0]/2,
                      -co_size[1]/2, co_size[1]/2),
              origin="lower"
              )
    ax.grid()

    fig.tight_layout()
    plt.show()

if show_midplane and show == "FIELD":
    fig, ax = plt.subplots(1, 1, figsize=(8.25, 8.25))
    print(field.shape)
    ax.imshow(field[:, :].T,
              cmap="binary_r",
              extent=(-co_size[0] / 2, co_size[0] / 2,
                      -co_size[1] / 2, co_size[1] / 2),
              origin="lower"
              )
    cs = ax.contour(coor[0].reshape(co_res_new[0], co_res_new[1]),
                    coor[1].reshape(co_res_new[0], co_res_new[1]),
                    field[:, :],
                    cmap="plasma_r")
    ax.clabel(cs, inline=True, fontsize=10)
    ax.grid()